# Generating names with recurrent neural networks

This time you'll understand thoroughly the insides of recurrent neural networks on a class of toy problems.

Struggle to find a name for the variable? Let's see how you'll come up with a name for your son/daughter.
Surely no human has expertize over what is a good child name, so let us train RNN instead ;)

You should fill in the gaps in the given function templates.

In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
import torch
from torch import nn
import torch.nn.functional as F
from tqdm.notebook import tqdm
from random import sample
from IPython.display import clear_output
import requests

%matplotlib inline

url = 'https://raw.githubusercontent.com/yandexdataschool/Practical_DL/fall19/week06_rnn/names'

response = requests.get(url)
with open('names', 'wb') as names_file:
    names_file.write(response.content)

### Our data
The dataset contains ~8k earthling names from different cultures, all in latin transcript.

This notebook has been designed so as to allow you to quickly swap names for something similar: deep learning article titles, IKEA furniture, pokemon names, etc.

In [ ]:
start_token = ' '

with open('names') as f:
    lines = f.read()[:-1].split('\n')
    lines = [start_token + line for line in lines]

In [ ]:
print ('n samples =', len(lines))
for x in lines[::1000]:
    print(x)

In [ ]:
MAX_LENGTH = max(map(len, lines))
print('max length = ', MAX_LENGTH)

plt.title('Sequence length distribution')
plt.hist(list(map(len, lines)),bins=25);

# Text processing

First we need next to collect a "vocabulary" of all unique tokens i.e. unique characters. We can then encode inputs as a sequence of character ids.

In [ ]:
#!source<mlpractice.rnn_torch.make_tokens>

In [ ]:
from mlpractice.tests.rnn_torch.test_make_tokens import test_all

test_all(make_tokens)

In [ ]:
#all unique characters go here
tokens = make_tokens('\n'.join(lines))

num_tokens = len(tokens)
print ('num_tokens = ', num_tokens)

assert 50 < num_tokens < 60, \
    "Names should contain within 50 and 60 unique tokens depending on encoding"

### Convert characters to integers

Torch is built for crunching numbers, not strings. 
To train our neural network, we'll need to replace characters with their indices in tokens list.

Let's compose a dictionary that does this mapping.

In [ ]:
#!source<mlpractice.rnn_torch.make_token_to_id>

In [ ]:
from mlpractice.tests.rnn_torch.test_make_token_to_id import test_all

test_all(make_token_to_id)

In [ ]:
token_to_id = make_token_to_id(tokens)

In [ ]:
def to_matrix(lines, max_len=None, pad=token_to_id[' '],
              dtype='int32', batch_first=True):
    r"""Casts a list of names into rnn-digestible matrix."""
    max_len = max_len or max(map(len, lines))
    lines_ix = np.zeros((len(lines), max_len), dtype) + pad

    for i in range(len(lines)):
        line_ix = [token_to_id[c] for c in lines[i]]
        lines_ix[i, :len(line_ix)] = line_ix

    if not batch_first: # convert [batch, time] into [time, batch]
        lines_ix = np.transpose(lines_ix)

    return lines_ix

In [ ]:
# Example: cast 4 random names to a matrix
print('\n'.join(lines[::2000]) + '\n')
print(to_matrix(lines[::2000]))

# Recurrent neural network

We can rewrite recurrent neural network as a consecutive application of dense layer to input $x_t$ and previous rnn state $h_t$. This is exactly what we're gonna do now.
<img src="./rnn.png" width=480>

Since we're training a language model, there should also be:
* An embedding layer that converts character id **x_t** to a vector
* An output layer that predicts probabilities of next phoneme

In [ ]:
#!source<mlpractice.rnn_torch.CharRNNCell>

In [ ]:
char_rnn = CharRNNCell(num_tokens)

### RNN loop

Once we've defined a single RNN step, we can apply it in a loop to get predictions on each step.

In [ ]:
#!source<mlpractice.rnn_torch.rnn_loop>

In [ ]:
batch_ix = to_matrix(lines[:5])
batch_ix = torch.tensor(batch_ix, dtype=torch.int64)

logp_seq = rnn_loop(char_rnn, batch_ix)

assert torch.max(logp_seq).data.numpy() <= 0
assert tuple(logp_seq.size()) ==  batch_ix.shape + (num_tokens,)

### Likelihood and gradients

We can now train our neural network to minimize crossentropy (maximize log-likelihood) with the actual next tokens.

To do so in a vectorized manner, we take `batch_ix[:, 1:]` — a matrix of token ids shifted one step to the left so i-th element is acutally the "next token" for i-th prediction

In [ ]:
predictions_logp = logp_seq[:, :-1]
actual_next_tokens = batch_ix[:, 1:]

logp_next = torch.gather(predictions_logp, dim=2, index=actual_next_tokens[:,:,None])

loss = -logp_next.mean()

In [ ]:
loss.backward()

In [ ]:
for w in char_rnn.parameters():
    assert w.grad is not None and torch.max(torch.abs(w.grad)).data.numpy() != 0, \
        f'Loss is not differentiable w.r.t. a weight with shape {(w.size(),)}. Check forward method.'

### The training loop

We train our char-rnn exactly the same way we train any deep learning model: by minibatch sgd.

The only difference is that this time we sample strings, not images or sound.

In [ ]:
char_rnn = CharRNNCell(num_tokens)
optimizer = torch.optim.Adam(char_rnn.parameters())
encoded_lines = to_matrix(lines, max_len=MAX_LENGTH)

In [ ]:
#!source<mlpractice.rnn_torch.train_rnn>

In [ ]:
history = train_rnn(encoded_lines, char_rnn, optimizer)

assert np.mean(history[:10]) > np.mean(history[-10:]), "RNN didn't converge."

### RNN: sampling
Once we've trained our network a bit, let's get to actually generating stuff. 
All we need is the single rnn step function you have defined in `char_rnn.forward`.

In [ ]:
def generate_sample(
    char_rnn,
    seed_phrase=' ',
    max_length=MAX_LENGTH,
    temperature=1.0,
):
    r"""The function generates text given a phrase
    of length at least SEQ_LENGTH.

    Parameters
    ----------
    char_rnn : nn.Module
        RNN model that will be used for generation.
    seed_phrase : str, optional
        Prefix characters. The RNN is asked to continue the phrase.
    max_length : int, optional
        Maximum output length, including seed_phrase.
    temperature : float, optional
        Coefficient for sampling. Higher temperature produces more chaotic
        outputs, smaller temperature converges to a single most likely output.
    """
    x_sequence = [token_to_id[token] for token in seed_phrase]
    x_sequence = torch.tensor([x_sequence], dtype=torch.int64)
    hid_state = char_rnn.initial_state(batch_size=1)
    
    # feed the seed phrase, if any
    for i in range(len(seed_phrase) - 1):
        hid_state, _ = char_rnn(x_sequence[:, i], hid_state)
    
    # start generating
    for _ in range(max_length - len(seed_phrase)):
        hid_state, logp_next = char_rnn(x_sequence[:, -1], hid_state)
        p_next = F.softmax(logp_next / temperature, dim=-1).data.numpy()[0]
        
        # sample next token and push it back into x_sequence
        next_ix = np.random.choice(num_tokens,p=p_next)
        next_ix = torch.tensor([[next_ix]], dtype=torch.int64)
        x_sequence = torch.cat([x_sequence, next_ix], dim=1)
        
    return ''.join([tokens[ix] for ix in x_sequence.data.numpy()[0]])

In [ ]:
for _ in range(20):
    print(generate_sample(char_rnn))

In [ ]:
for _ in range(20):
    print(generate_sample(char_rnn, seed_phrase=' Trump'))

### Try it out!
You've just implemented a recurrent language model that can be tasked with generating any kind of sequence, so there's plenty of data you can try it on:

* Novels/poems/songs of your favorite author
* News titles/clickbait titles
* Source code of Linux or Tensorflow
* Molecules in [smiles](https://en.wikipedia.org/wiki/Simplified_molecular-input_line-entry_system) format
* Melody in notes/chords format
* Ikea catalog titles
* Pokemon names
* Cards from Magic, the Gathering / Hearthstone

If you're willing to give it a try, here's what you wanna look at:
* Current data format is a sequence of lines, so a novel can be formatted as a list of sentences. Alternatively, you can change data preprocessing altogether.
* While some datasets are readily available, others can only be scraped from the web. Try `Selenium` or `Scrapy` for that.
* Make sure MAX_LENGTH is adjusted for longer datasets. There's also a bonus section about dynamic RNNs at the bottom.
* More complex tasks require larger RNN architecture, try more neurons or several layers. It would also require more training iterations.
* Long-term dependencies in music, novels or molecules are better handled with LSTM or GRU

__Good hunting!__

### More seriously

What we just did is a manual low-level implementation of RNN. While it's cool, i guess you won't like the idea of re-writing it from scratch on every occasion. 

As you might have guessed, torch has a solution for this. To be more specific, there are two options:
* `nn.RNNCell(emb_size, rnn_num_units)` — implements a single step of RNN just like you did. Basically concat-linear-tanh
* `nn.RNN(emb_size, rnn_num_units` — implements the whole rnn_loop for you.

There's also `nn.LSTMCell` vs `nn.LSTM`, `nn.GRUCell` vs `nn.GRU`, etc. etc.

In this example we'll rewrite the char_rnn and rnn_loop using high-level rnn API.

In [ ]:
class CharRNNLoop(nn.Module):
    def __init__(self, num_tokens=num_tokens, emb_size=16, rnn_num_units=64):
        super(self.__class__, self).__init__()
        self.emb = nn.Embedding(num_tokens, emb_size)
        self.rnn = nn.RNN(emb_size, rnn_num_units, batch_first=True)
        self.hid_to_logits = nn.Linear(rnn_num_units, num_tokens)
        
    def forward(self, x):
        h_seq, _ = self.rnn(self.emb(x))
        next_logits = self.hid_to_logits(h_seq)
        next_logp = F.log_softmax(next_logits, dim=-1)
        return next_logp
    
model = CharRNNLoop()

In [ ]:
# the model applies over the whole sequence
batch_ix = to_matrix(sample(lines, 32), max_len=MAX_LENGTH)
batch_ix = torch.tensor(batch_ix, dtype=torch.int64)

logp_seq = model(batch_ix)

# compute loss. This time we use nll_loss with some duct tape
loss = F.nll_loss(logp_seq[:, :-1].contiguous().view(-1, num_tokens), 
                  batch_ix[:, 1:].contiguous().view(-1))

loss.backward()

Here's another example:

In [ ]:
class CharLSTMCell(nn.Module):
    r"""Implements something like CharRNNCell, but with LSTM."""
    def __init__(self, num_tokens=len(tokens), embedding_size=16, rnn_num_units=64):
        super(self.__class__,self).__init__()
        self.num_units = rnn_num_units
        self.emb = nn.Embedding(num_tokens, embedding_size)
        self.lstm = nn.LSTMCell(embedding_size, rnn_num_units)
        self.rnn_to_logits = nn.Linear(rnn_num_units, num_tokens)
        
    def forward(self, x, prev_state):
        (prev_h, prev_c) = prev_state
        (next_h, next_c) = self.lstm(self.emb(x), (prev_h, prev_c))
        logits = self.rnn_to_logits(next_h)
        
        return (next_h, next_c), F.log_softmax(logits, -1)
    
    def initial_state(self, batch_size):
        """LSTM has two state variables, cell and hid."""
        return torch.zeros(batch_size, self.num_units), torch.zeros(batch_size, self.num_units)
    
char_lstm = CharLSTMCell()

In [ ]:
# the model applies over the whole sequence
batch_ix = to_matrix(sample(lines, 32), max_len=MAX_LENGTH)
batch_ix = torch.tensor(batch_ix, dtype=torch.int64)

logp_seq = rnn_loop(char_lstm, batch_ix)

# compute loss. This time we use nll_loss with some duct tape
loss = F.nll_loss(logp_seq[:, :-1].contiguous().view(-1, num_tokens), 
                  batch_ix[:, 1:].contiguous().view(-1))

loss.backward()

In [ ]:
# If you want to submit you solutions authorize
USERNAME = ""
PASSWORD = ""

from mlpractice.stats.stats_utils import _get_stats, submit

submit(USERNAME, PASSWORD, str(_get_stats()))

__Bonus quest:__ implement a model that uses 2 LSTM layers (the second lstm uses the first as input) and train it on your data.